In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

from webdriver_manager.chrome import ChromeDriverManager

def scrape_job_offers(url):
    # Setup Chrome options
    chrome_options = Options()
    # Runs Chrome in headless mode.
    chrome_options.add_argument("--headless")

    # Setup Selenium with ChromeDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Navigate to the page
    driver.get(url)
    
    # List of offers on the left side
    job_list_selector = '#jsJobResContent .jobItem'
    # Selector for the div where offer details are displayed
    job_detail_title = '.jobResultsTitle'
    job_detail_salary = '.jobResultsSalary'
    job_detail_loc = '.jobResultsLoc'
    job_detail_type = '.jobResultsType'
    job_detail_when = '.when'

    #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Find all job offer items in the list
    job_items = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, job_list_selector)))
    print(f"Job_tems found: {len(job_items)}")
    
    for index, job_item in enumerate(job_items):
        # Click on the job item to load its details into the div; may need to scroll into view
        ActionChains(driver).move_to_element(job_item).click(job_item).perform()

        #JobDetailContainer        
            ### Right panel containing all job details: including description, location, industry
        
        #md_skills
            ### Job description

        #md_items
            ### All the items after the description
            ###    md_location, md_industry, md_category, md_duration, md_start_date, md_rate, md_recruiter(a-link), md_ref, md_posted_date, md_permalink (a-link)
        
        # Extract details from the div
        title = job_item.find_element(By.CSS_SELECTOR, job_detail_title).text
        salary = job_item.find_element(By.CSS_SELECTOR, job_detail_salary).text
        loc = job_item.find_element(By.CSS_SELECTOR, job_detail_loc).text
        type = job_item.find_element(By.CSS_SELECTOR, job_detail_type).text
        when = job_item.find_element(By.CSS_SELECTOR, job_detail_when).text
        
        print(f"*Job offer {index + 1}:\n{title}\n{salary}\n{loc}\n{type}\n{when}\n")
        
        # It might be necessary to navigate back to the list or re-find elements if the DOM changes
        # In some cases, you might need to re-open the job list page or re-find the job_items list due to stale elements
        
    # Close the browser
    driver.quit()

if __name__ == "__main__":
    url = "https://www.jobserve.com/gb/en/JobSearch.aspx?shid=2981D9787360CBB0C8BE"
    scrape_job_offers(url)
    print("Ended")


Job_tems found: 25
Machine Learning Engineer with AWS - London
800-1k per day
London
Contract
3 days ago
Details for job offer 1:
Machine Learning Engineer with AWS - London
800-1k per day
London
Contract
3 days ago

Senior Machine Learning Engineer - Generative AI - Inside IR35 - US Time Zone
London/Remote
Contract
6 days ago
Details for job offer 2:
Senior Machine Learning Engineer - Generative AI - Inside IR35 - US Time Zone

London/Remote
Contract
6 days ago

Python Developer Software Engineer React Finance Trading London
£100k - £120k per annum + 20% bonus + 10% Pension
London
Permanent
2 days ago
Details for job offer 3:
Python Developer Software Engineer React Finance Trading London
£100k - £120k per annum + 20% bonus + 10% Pension
London
Permanent
2 days ago

Python Developer Software Engineer React Finance Trading London
£80k - £100k per annum + 15% bonus + 10% Pension
London
Permanent
2 days ago
Details for job offer 4:
Python Developer Software Engineer React Finance Trading